<a href="https://colab.research.google.com/github/Aniket-tempest/HAR-Employee-Identification/blob/main/HAR_DeepLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from tensorflow.keras import backend as K

In [4]:
np.random.seed(42)
tf.random.set_seed(42)

In [5]:
from keras import backend as K
from keras.models import Sequential
from keras.layers import LSTM, TimeDistributed, Conv1D, MaxPooling1D, Flatten
from keras.layers.core import Dense, Dropout

In [6]:
DATADIR = 'UCI HAR Dataset'

In [7]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [8]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the signals
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'drive/MyDrive/{DATADIR}/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).to_numpy()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [9]:
def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'drive/MyDrive/{DATADIR}/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).to_numpy()

In [10]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [29]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 32

In [30]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [31]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

In [32]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))
print(n_classes)

128
9
7352
6


In [15]:
X_train.shape

(7352, 128, 9)

In [33]:
# Initiliazing the sequential model
model1 = Sequential()
# Configuring the parameters
model1.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model1.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model1.add(Dense(n_classes, activation='sigmoid'))
model1.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 32)                5376      
                                                                 
 dropout_3 (Dropout)         (None, 32)                0         
                                                                 
 dense_3 (Dense)             (None, 6)                 198       
                                                                 
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [34]:
# Compiling the model
model1.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [35]:
# Training the model
model1.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Epoch 1/30
460/460 [==============================] - 27s 53ms/step - loss: 1.0783 - accuracy: 0.5540 - val_loss: 0.9257 - val_accuracy: 0.6206
Epoch 2/30
460/460 [==============================] - 25s 54ms/step - loss: 0.7802 - accuracy: 0.6491 - val_loss: 0.7082 - val_accuracy: 0.7068
Epoch 3/30
460/460 [==============================] - 25s 54ms/step - loss: 0.6645 - accuracy: 0.7255 - val_loss: 0.6880 - val_accuracy: 0.7370
Epoch 4/30
460/460 [==============================] - 24s 52ms/step - loss: 0.5972 - accuracy: 0.7584 - val_loss: 0.5995 - val_accuracy: 0.7713
Epoch 5/30
460/460 [==============================] - 26s 56ms/step - loss: 0.5267 - accuracy: 0.7969 - val_loss: 0.5931 - val_accuracy: 0.7808
Epoch 6/30
460/460 [==============================] - 26s 57ms/step - loss: 0.4502 - accuracy: 0.8424 - val_loss: 0.5429 - val_accuracy: 0.8256
Epoch 7/30
460/460 [==============================] - 26s 56ms/step - loss: 0.3464 - accuracy: 0.8942 - val_loss: 0.4496 - val_accuracy:

In [36]:
# Training Evaluation
print("LSTM Training Accuracy")
score = model1.evaluate(X_train, Y_train)
print("Accuracy: ", score[1])
print("Loss: ", score[0])
print(confusion_matrix(np.argmax(Y_train, axis=1), np.argmax(model1.predict(X_train), axis=1)))

LSTM Training Accuracy
230/230 [==============================] - 3s 13ms/step - loss: 0.2447 - accuracy: 0.9283
Accuracy:  0.9283187985420227
Loss:  0.24472583830356598
230/230 [==============================] - 3s 13ms/step
[[1224    2    0    0    0    0]
 [   0 1073    0    0    0    0]
 [   0    5  981    0    0    0]
 [   0    0    0 1206   80    0]
 [   0    0    0  257 1117    0]
 [   0    0    0  183    0 1224]]


In [37]:
# Testing
print("LSTM Testing Accuracy")
score = model1.evaluate(X_test, Y_test)
print("Accuracy: ", score[1])
print("Loss: ", score[0])
print(confusion_matrix(np.argmax(Y_test, axis=1), np.argmax(model1.predict(X_test), axis=1)))

LSTM Testing Accuracy
93/93 [==============================] - 1s 14ms/step - loss: 0.5050 - accuracy: 0.8965
Accuracy:  0.8965049386024475
Loss:  0.5050035715103149
93/93 [==============================] - 1s 13ms/step
[[472  15   7   1   1   0]
 [  1 465   3   1   0   1]
 [  2  24 392   1   0   1]
 [  0   0   0 421  70   0]
 [  1   1   0 109 421   0]
 [  0   0   0  39  27 471]]


In [44]:
tf.keras.backend.clear_session()

In [45]:
n_steps, n_length = 4, 32
X_train = X_train.reshape((X_train.shape[0], n_steps, n_length, input_dim))
X_test = X_test.reshape((X_test.shape[0], n_steps, n_length, input_dim))

In [46]:
# define model
# CNN layers for feature extraction on input data combined with LSTMs to support sequence prediction
model2 = Sequential()
# CNN layers
model2.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu'), input_shape=(None,n_length,input_dim)))
model2.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')))
model2.add(TimeDistributed(Dropout(0.5)))
model2.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model2.add(TimeDistributed(Flatten()))
# LSTM layer
model2.add(LSTM(n_hidden))
model2.add(Dropout(0.5))
model2.add(Dense(n_hidden, activation='relu'))
model2.add(Dense(n_classes, activation='sigmoid'))
model2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDistr  (None, None, 30, 64)     1792      
 ibuted)                                                         
                                                                 
 time_distributed_1 (TimeDis  (None, None, 28, 64)     12352     
 tributed)                                                       
                                                                 
 time_distributed_2 (TimeDis  (None, None, 28, 64)     0         
 tributed)                                                       
                                                                 
 time_distributed_3 (TimeDis  (None, None, 14, 64)     0         
 tributed)                                                       
                                                                 
 time_distributed_4 (TimeDis  (None, None, 896)        0

In [40]:
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [41]:
# Training the model
model2.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Epoch 1/30
460/460 [==============================] - 47s 97ms/step - loss: 0.5374 - accuracy: 0.7828 - val_loss: 0.4242 - val_accuracy: 0.8582
Epoch 2/30
460/460 [==============================] - 10s 23ms/step - loss: 0.2019 - accuracy: 0.9294 - val_loss: 0.2971 - val_accuracy: 0.9006
Epoch 3/30
460/460 [==============================] - 10s 21ms/step - loss: 0.1490 - accuracy: 0.9444 - val_loss: 0.3459 - val_accuracy: 0.8972
Epoch 4/30
460/460 [==============================] - 10s 21ms/step - loss: 0.1312 - accuracy: 0.9467 - val_loss: 0.3414 - val_accuracy: 0.8996
Epoch 5/30
460/460 [==============================] - 10s 21ms/step - loss: 0.1264 - accuracy: 0.9502 - val_loss: 0.5524 - val_accuracy: 0.8504
Epoch 6/30
460/460 [==============================] - 10s 21ms/step - loss: 0.1163 - accuracy: 0.9502 - val_loss: 0.3347 - val_accuracy: 0.9094
Epoch 7/30
460/460 [==============================] - 11s 24ms/step - loss: 0.1129 - accuracy: 0.9516 - val_loss: 0.4192 - val_accuracy:

In [42]:
# Training evaluation
print("CNN-LSTM Training Accuracy")
score = model2.evaluate(X_train, Y_train)
print("Accuracy: ", score[1])
print("Loss: ", score[0])
print(confusion_matrix(np.argmax(Y_train, axis=1), np.argmax(model2.predict(X_train), axis=1)))

CNN-LSTM Training Accuracy
230/230 [==============================] - 2s 8ms/step - loss: 0.1060 - accuracy: 0.9614
Accuracy:  0.9613710641860962
Loss:  0.10595442354679108
230/230 [==============================] - 3s 9ms/step
[[1226    0    0    0    0    0]
 [  63 1010    0    0    0    0]
 [   2    0  984    0    0    0]
 [   2    0    0 1218   66    0]
 [   0    0    0  151 1223    0]
 [   0    0    0    0    0 1407]]


In [43]:
# Testing
print("CNN-LSTM Testing Accuracy")
score = model2.evaluate(X_test, Y_test)
print("Accuracy: ", score[1])
print("Loss: ", score[0])
print(confusion_matrix(np.argmax(Y_test, axis=1), np.argmax(model2.predict(X_test), axis=1)))

CNN-LSTM Testing Accuracy
93/93 [==============================] - 1s 9ms/step - loss: 0.8079 - accuracy: 0.8633
Accuracy:  0.8632507920265198
Loss:  0.8079065680503845
93/93 [==============================] - 1s 11ms/step
[[472   0  24   0   0   0]
 [127 312  32   0   0   0]
 [  9   0 411   0   0   0]
 [  4  24   0 423  40   0]
 [  1   0   0 122 409   0]
 [  0  20   0   0   0 517]]
